# 1. Go through https://aeturrell.github.io/coding-for-economists/data-extraction.html up to 'Extracting data from PDFs' section. Recreate the code on your  machine.

In [2]:
import fitz #Couldn't be read with the pdftotext function because of the version, so it's replaced with the PyMuPDF
from pathlib import Path
 

pdf_path = Path("data/pdf_with_table.pdf")  

try:
    pdf_document = fitz.open(pdf_path)  
    all_text = "" 
 
    for page_num in range(pdf_document.page_count):
        page = pdf_document.load_page(page_num) 
        page_text = page.get_text()  
        all_text += page_text + "\n\n"  
    
    print(all_text[:220])  
 
finally:

    pdf_document.close() 
 

3 
 
2 Quantifying Fuel-Saving Opportunities from Specific Driving 
Behavior Changes 
2.1 
Savings from Improving Individual Driving Profiles 
2.1.1 Drive Profile Subsample from Real-World Travel Survey 
The interim repo


In [1]:
import pdfplumber #Changed functions due to system version
 
with pdfplumber.open('data/pdf_with_table.pdf') as pdf:

    for page in pdf.pages:   
        tables = page.extract_tables()
        table = tables[0]
        for row in table:
            print(row)

['Cycle\nName', 'KI\n(1/km)', 'Distance\n(mi)', 'Percent Fuel Savings', None, None, None]
[None, None, None, 'Improved\nSpeed', 'Decreased\nAccel', 'Eliminate\nStops', 'Decreased\nIdle']
['2012_2', '3.30', '1.3', '5.9%', '9.5%', '29.2%', '17.4%']
['2145_1', '0.68', '11.2', '2.4%', '0.1%', '9.5%', '2.7%']
['4234_1', '0.59', '58.7', '8.5%', '1.3%', '8.5%', '3.3%']
['2032_2', '0.17', '57.8', '21.7%', '0.3%', '2.7%', '1.2%']
['4171_1', '0.07', '173.9', '58.1%', '1.6%', '2.1%', '0.5%']


In [ ]:
#import textract
#text = textract.process(Path('path/to/file.extension'))
#Currently, this third-party library does not run on Windows.

# 2.Go through https://www.geeksforgeeks.org/scrape-imdb-movie-rating-and-detalls-using-python/. Recreate the code on your machine.

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from time import sleep

In [ ]:
headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/116.0',
    'Accept': '*/*',
    'Accept-Language': 'zh-CN,zh;q=0.8,zh-TW;q=0.7,zh-HK;q=0.5,en-US;q=0.3,en;q=0.2',
    'Accept-Encoding': 'gzip, deflate, br',
    'Content-Type': 'text/plain;charset=UTF-8',
}

url = 'https://www.imdb.com/chart/top'
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")
movies = soup.select('li.ipc-metadata-list-summary-item')

In [ ]:
index = 0

title_string = movies[index].select('h3')[0].text.split(". ")
place = title_string[0]
movie_title = title_string[1]
rating = movies[index].select('span.ipc-rating-star--rating')[0].text
year = movies[index].select('span.cli-title-metadata-item')[0].text
title_id_href = movies[0].select('a.ipc-lockup-overlay')[0].get('href')
title_id = title_id_href.split("/")[2]
stats_url = 'https://caching.graphql.imdb.com/?operationName=Title_Summary_Prompt_From_Base&variables={"id":"' + title_id + '","locale":"zh-CN","location":{"latLong":{"lat":"22.3","long":"114.2"}},"promotedProvider":null,"providerId":null}&extensions={"persistedQuery":{"sha256Hash":"28fdb27482f0852bc70811f36a5fab72e9afd1589b37c5e1aaecf438d418f6c7","version":1}}'
stats_response = requests.get(stats_url, headers=headers)
stats_json_data = stats_response.json()
stats = ', '.join([item['name']['nameText']['text'] for item in stats_json_data['data']['title']['principalCast'][0]['credits']])
stats_json_data

In [ ]:
movie_data_list = []

for index in range(len(movies)):
	title_string = movies[index].select('h3')[0].text.split(". ")
	place = title_string[0]
	movie_title = title_string[1]
	rating = movies[index].select('span.ipc-rating-star--rating')[0].text
	year = movies[index].select('span.cli-title-metadata-item')[0].text
	title_id_href = movies[index].select('a.ipc-lockup-overlay')[0].get('href')
	title_id = title_id_href.split("/")[2]
	stats_url = 'https://caching.graphql.imdb.com/?operationName=Title_Summary_Prompt_From_Base&variables={"id":"' + title_id + '","locale":"zh-CN","location":{"latLong":{"lat":"22.3","long":"114.2"}},"promotedProvider":null,"providerId":null}&extensions={"persistedQuery":{"sha256Hash":"28fdb27482f0852bc70811f36a5fab72e9afd1589b37c5e1aaecf438d418f6c7","version":1}}'
	stats_response = requests.get(stats_url, headers=headers)
	stats_json_data = stats_response.json()
	stats = ', '.join([item['name']['nameText']['text'] for item in stats_json_data['data']['title']['principalCast'][0]['credits']])

	data = {"place": place,
			"movie_title": movie_title,
			"rating": rating,
			"year": year,
			"star_cast": stats,
			}
	movie_data_list.append(data)
	sleep(0.2)

In [ ]:
df = pd.DataFrame(movie_data_list)
df.to_csv('data/imdb_top_250_movies.csv',index=False)

In [ ]:
df

# 3.scrape data from hitps:/movie.douban.com/top250 and make visualizations.Use hitps:/blog,csdn.net/9g 5174909/article/detais/143106786 as an example

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
 
douban_url_template = "https://movie.douban.com/top250?start={}"
douban_headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/116.0',
}
 
douban_movies = []

In [ ]:
# 分页
for start in range(0, 250, 25):
    douban_url = douban_url_template.format(start)
    douban_response = requests.get(douban_url, headers=douban_headers)
    douban_response.encoding = 'utf-8'  # 设置编码方式
    douban_soup = BeautifulSoup(douban_response.text, 'html.parser')
 
    for item in douban_soup.find_all('div', class_='item'):
        title = item.find('span', class_='title').get_text()  # 电影名称
        
        # 获取年份信息
        year_info = item.find('p').find('span', class_='pl')
        if year_info:
            year = year_info.get_text().strip().replace('(', '').replace(')', '')  # 清理括号
        else:
            year = '未知'
 
        description_span = item.find('span', class_='inq')  # 电影描述
        rating = item.find('span', class_='rating_num').get_text()  # 评分
        
        try:
            votes = item.find('div', class_='star').find_all('span')[3].get_text()  # 评价人数
        except IndexError:
            # 如果找不到，可以尝试其他方式或者设置为None/默认值
            votes = '未知'
        
        # 如果没有描述，将其置为空字符串
        description = description_span.get_text() if description_span else ''
        
        movie = {
            "title": title,
            "year": year,
            "description": description,
            "rating": rating,
            "votes": votes.replace('人评价', '').strip() if votes else '0'  # 如果没有votes，则设置为0或其他默认值
        }
        douban_movies.append(movie)
 


In [ ]:
douban_df = pd.DataFrame(douban_movies)
douban_df.to_csv('data/douban_top250.csv', index=False)

In [ ]:
douban_df